In [9]:
#imports 
import json
import requests
import pandas as pd
import matplotlib as plt
import pprint as pprint
from pandas.io.json import json_normalize
import gmaps
import gmaps.datasets

In [10]:
# data file to be saved and pulled from
rawData = "../data/rawData/master_normalized_raw_dataframe.csv"
# renamed data file to be saved and pulled from
orgRawData = "../data/rawData/master_organized_raw_dataframe.csv"
# No missing data file to be saved and pulled from
noMisingData = "../data/rawData/master_no_missing_raw_data.csv"
# Raw geo data file to be saved and pulled from
rawGeoData = "../data/rawData/master_geo_raw_dataframe.csv"

# path csv file location
pathRawData = "../data/rawData/"

In [11]:
# Remember to update the config file with your API key
from config import api_key
from config import api_id
from config import gkey

In [12]:
url = "https://api.adzuna.com/v1/api/jobs/us/search/"
#have to create iteration to get all pages of data
page = "1"
api_details ="?" + "app_id=" + api_id + "&app_key=" + api_key
# Build query URL
query_url = url + page + api_details
query_url

'https://api.adzuna.com/v1/api/jobs/us/search/1?app_id=a9ccf687&app_key=40f9330010ceb8d80cf656346e80a002'

In [13]:
data_response = requests.get(query_url)
data_json = data_response.json()
type(data_json)


dict

In [ ]:
#Getting data using json_normalize
#LINK: https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.io.json.json_normalize.html
#Page 33683 iteration complete
pages_test = [str(x) for x in range(11967,55001)]

counter = 0
# Build query URL
data_pages_df = pd.DataFrame()

for page_num in pages_test:
    try:
        response = requests.get(url + page_num + api_details).json()

        #create a df to store the normalized page that has the job postings
        normalized_page = json_normalize(response['results'])

        #add normalized data into a df
        data_page_df = pd.DataFrame(normalized_page)

        #add the url column to df incase an error occurs you know what page you were on
        data_page_df['query_url'] = 'page number ' + page_num

        #append to a new df so each page can be saved
        #Sorting because non-concatenation axis is not aligned
        data_pages_df = data_pages_df.append(data_page_df, sort=True)

        counter += 1
        print(f"Page {page_num} iteration complete")
    except ValueError:  # includes simplejson.decoder.JSONDecodeError
        print('JSONDecodeError occured. Skipping...')
print(f'total rows iterated | {counter * 10}')   
      

Page 11967 iteration complete
Page 11968 iteration complete
Page 11969 iteration complete
Page 11970 iteration complete
Page 11971 iteration complete
Page 11972 iteration complete
Page 11973 iteration complete
Page 11974 iteration complete
Page 11975 iteration complete
Page 11976 iteration complete
Page 11977 iteration complete
Page 11978 iteration complete
Page 11979 iteration complete
Page 11980 iteration complete
Page 11981 iteration complete
Page 11982 iteration complete
Page 11983 iteration complete
Page 11984 iteration complete
Page 11985 iteration complete
Page 11986 iteration complete
Page 11987 iteration complete
Page 11988 iteration complete
Page 11989 iteration complete
Page 11990 iteration complete
Page 11991 iteration complete
Page 11992 iteration complete
Page 11993 iteration complete
Page 11994 iteration complete
Page 11995 iteration complete
Page 11996 iteration complete
Page 11997 iteration complete
Page 11998 iteration complete
Page 11999 iteration complete
Page 12000

Page 12241 iteration complete
Page 12242 iteration complete
Page 12243 iteration complete
Page 12244 iteration complete
Page 12245 iteration complete
Page 12246 iteration complete
Page 12247 iteration complete
Page 12248 iteration complete
Page 12249 iteration complete
Page 12250 iteration complete
Page 12251 iteration complete
Page 12252 iteration complete
Page 12253 iteration complete
Page 12254 iteration complete
Page 12255 iteration complete
Page 12256 iteration complete
Page 12257 iteration complete
Page 12258 iteration complete
Page 12259 iteration complete
Page 12260 iteration complete
Page 12261 iteration complete
Page 12262 iteration complete
Page 12263 iteration complete
Page 12264 iteration complete
Page 12265 iteration complete
Page 12266 iteration complete
Page 12267 iteration complete
Page 12268 iteration complete
Page 12269 iteration complete
Page 12270 iteration complete
Page 12271 iteration complete
Page 12272 iteration complete
Page 12273 iteration complete
Page 12274

Page 12515 iteration complete
Page 12516 iteration complete
Page 12517 iteration complete
Page 12518 iteration complete
Page 12519 iteration complete
Page 12520 iteration complete
Page 12521 iteration complete
Page 12522 iteration complete
Page 12523 iteration complete
Page 12524 iteration complete
Page 12525 iteration complete
Page 12526 iteration complete
Page 12527 iteration complete
Page 12528 iteration complete
Page 12529 iteration complete
Page 12530 iteration complete
Page 12531 iteration complete
Page 12532 iteration complete
Page 12533 iteration complete
Page 12534 iteration complete
Page 12535 iteration complete
Page 12536 iteration complete
Page 12537 iteration complete
Page 12538 iteration complete
Page 12539 iteration complete
Page 12540 iteration complete
Page 12541 iteration complete
Page 12542 iteration complete
Page 12543 iteration complete
Page 12544 iteration complete
Page 12545 iteration complete
Page 12546 iteration complete
Page 12547 iteration complete
Page 12548

Page 12788 iteration complete
Page 12789 iteration complete
Page 12790 iteration complete
Page 12791 iteration complete
Page 12792 iteration complete
Page 12793 iteration complete
Page 12794 iteration complete
Page 12795 iteration complete
Page 12796 iteration complete
Page 12797 iteration complete
Page 12798 iteration complete
Page 12799 iteration complete
Page 12800 iteration complete
Page 12801 iteration complete
Page 12802 iteration complete
Page 12803 iteration complete
Page 12804 iteration complete
Page 12805 iteration complete
Page 12806 iteration complete
Page 12807 iteration complete
Page 12808 iteration complete
Page 12809 iteration complete
Page 12810 iteration complete
Page 12811 iteration complete
Page 12812 iteration complete
Page 12813 iteration complete
Page 12814 iteration complete
Page 12815 iteration complete
Page 12816 iteration complete
Page 12817 iteration complete
Page 12818 iteration complete
Page 12819 iteration complete
Page 12820 iteration complete
Page 12821

Page 13062 iteration complete
Page 13063 iteration complete
Page 13064 iteration complete
Page 13065 iteration complete
Page 13066 iteration complete
Page 13067 iteration complete
Page 13068 iteration complete
Page 13069 iteration complete
Page 13070 iteration complete
Page 13071 iteration complete
Page 13072 iteration complete
Page 13073 iteration complete
Page 13074 iteration complete
Page 13075 iteration complete
Page 13076 iteration complete
Page 13077 iteration complete
Page 13078 iteration complete
Page 13079 iteration complete
Page 13080 iteration complete
Page 13081 iteration complete
Page 13082 iteration complete
Page 13083 iteration complete
Page 13084 iteration complete
Page 13085 iteration complete
Page 13086 iteration complete
Page 13087 iteration complete
Page 13088 iteration complete
Page 13089 iteration complete
Page 13090 iteration complete
Page 13091 iteration complete
Page 13092 iteration complete
Page 13093 iteration complete
Page 13094 iteration complete
Page 13095

...

Page 32514 iteration complete
Page 32515 iteration complete
Page 32516 iteration complete
Page 32517 iteration complete
Page 32518 iteration complete
Page 32519 iteration complete
Page 32520 iteration complete
Page 32521 iteration complete
Page 32522 iteration complete
Page 32523 iteration complete
Page 32524 iteration complete
Page 32525 iteration complete
Page 32526 iteration complete
Page 32527 iteration complete
Page 32528 iteration complete
Page 32529 iteration complete
Page 32530 iteration complete
Page 32531 iteration complete
Page 32532 iteration complete
Page 32533 iteration complete
Page 32534 iteration complete
Page 32535 iteration complete
Page 32536 iteration complete
Page 32537 iteration complete
Page 32538 iteration complete
Page 32539 iteration complete
Page 32540 iteration complete
Page 32541 iteration complete
Page 32542 iteration complete
Page 32543 iteration complete
Page 32544 iteration complete
Page 32545 iteration complete
Page 32546 iteration complete
Page 32547

Page 32788 iteration complete
Page 32789 iteration complete
Page 32790 iteration complete
Page 32791 iteration complete
Page 32792 iteration complete
Page 32793 iteration complete
Page 32794 iteration complete
Page 32795 iteration complete
Page 32796 iteration complete
Page 32797 iteration complete
Page 32798 iteration complete
Page 32799 iteration complete
Page 32800 iteration complete
Page 32801 iteration complete
Page 32802 iteration complete
Page 32803 iteration complete
Page 32804 iteration complete
Page 32805 iteration complete
Page 32806 iteration complete
Page 32807 iteration complete
Page 32808 iteration complete
Page 32809 iteration complete
Page 32810 iteration complete
Page 32811 iteration complete
Page 32812 iteration complete
Page 32813 iteration complete
Page 32814 iteration complete
Page 32815 iteration complete
Page 32816 iteration complete
Page 32817 iteration complete
Page 32818 iteration complete
Page 32819 iteration complete
Page 32820 iteration complete
Page 32821

Page 33062 iteration complete
Page 33063 iteration complete
Page 33064 iteration complete
Page 33065 iteration complete
Page 33066 iteration complete
Page 33067 iteration complete
Page 33068 iteration complete
Page 33069 iteration complete
Page 33070 iteration complete
Page 33071 iteration complete
Page 33072 iteration complete
Page 33073 iteration complete
Page 33074 iteration complete
Page 33075 iteration complete
Page 33076 iteration complete
Page 33077 iteration complete
Page 33078 iteration complete
Page 33079 iteration complete
Page 33080 iteration complete
Page 33081 iteration complete
Page 33082 iteration complete
Page 33083 iteration complete
Page 33084 iteration complete
Page 33085 iteration complete
Page 33086 iteration complete
Page 33087 iteration complete
Page 33088 iteration complete
Page 33089 iteration complete
Page 33090 iteration complete
Page 33091 iteration complete
Page 33092 iteration complete
Page 33093 iteration complete
Page 33094 iteration complete
Page 33095

Page 33336 iteration complete
Page 33337 iteration complete
Page 33338 iteration complete
Page 33339 iteration complete
Page 33340 iteration complete
Page 33341 iteration complete
Page 33342 iteration complete
Page 33343 iteration complete
Page 33344 iteration complete
Page 33345 iteration complete
Page 33346 iteration complete
Page 33347 iteration complete
Page 33348 iteration complete
Page 33349 iteration complete
Page 33350 iteration complete
Page 33351 iteration complete
Page 33352 iteration complete
Page 33353 iteration complete
Page 33354 iteration complete
Page 33355 iteration complete
Page 33356 iteration complete
Page 33357 iteration complete
Page 33358 iteration complete
Page 33359 iteration complete
Page 33360 iteration complete
Page 33361 iteration complete
Page 33362 iteration complete
Page 33363 iteration complete
Page 33364 iteration complete
Page 33365 iteration complete
Page 33366 iteration complete
Page 33367 iteration complete
Page 33368 iteration complete
Page 33369

Page 33609 iteration complete
Page 33610 iteration complete
Page 33611 iteration complete
Page 33612 iteration complete
Page 33613 iteration complete
Page 33614 iteration complete
Page 33615 iteration complete
Page 33616 iteration complete
Page 33617 iteration complete
Page 33618 iteration complete
Page 33619 iteration complete
Page 33620 iteration complete
Page 33621 iteration complete
Page 33622 iteration complete
Page 33623 iteration complete
Page 33624 iteration complete
Page 33625 iteration complete
Page 33626 iteration complete
Page 33627 iteration complete
Page 33628 iteration complete
Page 33629 iteration complete
Page 33630 iteration complete
Page 33631 iteration complete
Page 33632 iteration complete
Page 33633 iteration complete
Page 33634 iteration complete
Page 33635 iteration complete
Page 33636 iteration complete
Page 33637 iteration complete
Page 33638 iteration complete
Page 33639 iteration complete
Page 33640 iteration complete
Page 33641 iteration complete
Page 33642

In [3]:
#save raw data to dataRaw folder
data_pages_df.to_csv(rawData, index=False)

NameError: name 'data_pages_df' is not defined

In [ ]:
#calling the csv raw data
csv_file = pd.read_csv(rawData)
csv_file.head(3)


In [ ]:
#making a copy
csv_file_copy = csv_file.copy()

#getting only the rows that we are using
csv_file_copy = csv_file_copy[["id","title","category.label","company.display_name", "location.display_name", "latitude","longitude" ]]
csv_file_copy.head()

In [148]:
#Renaming the column names

renamed_csv_file = csv_file_copy.rename(columns = {
    "id": "Job Posting ID",
    "title": "Job Title",
    "category.label":"Category",
    "company.display_name":"Company Name",
    "location.display_name":"Location",
    "latitude":"Lat",
    "longitude":"Lng"
})
renamed_csv_file.head()

,Job Posting ID,Job Title,Category,Company Name,Location,Lat,Lng
0,1312805093,Physician: Family Physician Job Opening in Ups...,Healthcare & Nursing Jobs,CompHealth,"Clayton, Jefferson County",44.239491,-76.085776
1,1312805901,Physician: Urology Need in Western Ohio with L...,Healthcare & Nursing Jobs,CompHealth,"Ohio, US",40.305232,-82.824785
2,1328241931,Physician: Internal Medicine Physician Job in ...,Healthcare & Nursing Jobs,CompHealth,"Cheshire, New Haven County",41.508367,-72.910620
3,1334502463,Physician: A Locums Otolaryngologist Is Wanted...,Healthcare & Nursing Jobs,CompHealth,"Marion, Williamson County",37.730605,-88.933126
4,1312805719,"Physician: Hospitalist needed in Hartford, CT ...",Healthcare & Nursing Jobs,CompHealth,"Connecticut, US",41.601859,-72.657316


In [149]:
#save raw data to dataRaw folder
renamed_csv_file.to_csv(orgRawData, index=False)

In [150]:
#calling the csv org data
org_raw_csv_file = pd.read_csv(orgRawData)
org_raw_csv_file_copy = org_raw_csv_file.copy()
org_raw_csv_file_copy.head()

,Job Posting ID,Job Title,Category,Company Name,Location,Lat,Lng
0,1312805093,Physician: Family Physician Job Opening in Ups...,Healthcare & Nursing Jobs,CompHealth,"Clayton, Jefferson County",44.239491,-76.085776
1,1312805901,Physician: Urology Need in Western Ohio with L...,Healthcare & Nursing Jobs,CompHealth,"Ohio, US",40.305232,-82.824785
2,1328241931,Physician: Internal Medicine Physician Job in ...,Healthcare & Nursing Jobs,CompHealth,"Cheshire, New Haven County",41.508367,-72.910620
3,1334502463,Physician: A Locums Otolaryngologist Is Wanted...,Healthcare & Nursing Jobs,CompHealth,"Marion, Williamson County",37.730605,-88.933126
4,1312805719,"Physician: Hospitalist needed in Hartford, CT ...",Healthcare & Nursing Jobs,CompHealth,"Connecticut, US",41.601859,-72.657316


In [151]:
#Count Rows of all columns
org_raw_csv_file_copy.count()

Job Posting ID    1000
Job Title         1000
Category          1000
Company Name      1000
Location          1000
Lat                983
Lng                983
dtype: int64

In [152]:
#Delete Rows with missing data from lat and lng
org_data = org_raw_csv_file_copy.dropna()
org_data.count()

Job Posting ID    983
Job Title         983
Category          983
Company Name      983
Location          983
Lat               983
Lng               983
dtype: int64

In [153]:
#Save organized raw data into rawData folder
org_data.to_csv(noMisingData, index=False)

In [154]:
#calling the csv org data
no_missing_data_csv_file = pd.read_csv(noMisingData)
no_missing_data = no_missing_data_csv_file.copy()
no_missing_data.head()

,Job Posting ID,Job Title,Category,Company Name,Location,Lat,Lng
0,1312805093,Physician: Family Physician Job Opening in Ups...,Healthcare & Nursing Jobs,CompHealth,"Clayton, Jefferson County",44.239491,-76.085776
1,1312805901,Physician: Urology Need in Western Ohio with L...,Healthcare & Nursing Jobs,CompHealth,"Ohio, US",40.305232,-82.824785
2,1328241931,Physician: Internal Medicine Physician Job in ...,Healthcare & Nursing Jobs,CompHealth,"Cheshire, New Haven County",41.508367,-72.910620
3,1334502463,Physician: A Locums Otolaryngologist Is Wanted...,Healthcare & Nursing Jobs,CompHealth,"Marion, Williamson County",37.730605,-88.933126
4,1312805719,"Physician: Hospitalist needed in Hartford, CT ...",Healthcare & Nursing Jobs,CompHealth,"Connecticut, US",41.601859,-72.657316


In [155]:
#Creating new column for combined LatLng
no_missing_data['LatLng'] = no_missing_data['Lat'].map(str) + "," + no_missing_data['Lng'].map(str)

LatLng = list(no_missing_data['LatLng'])
LatLng

['44.239490999999994,-76.085776',
 '40.305232000000004,-82.82478499999998',
 '41.508367,-72.91062',
 '37.730605,-88.933126',
 '41.60185900000001,-72.65731600000002',
 '44.022931,-73.145015',
 '33.160641999999996,-117.238216',
 '40.621455,-79.152535',
 '47.515458,-111.28035700000001',
 '40.305232000000004,-82.82478499999998',
 '46.817181,-95.845325',
 '42.104245,-72.318952',
 '43.639036,-72.922874',
 '34.213396,-79.820616',
 '39.863792,-89.15483',
 '41.032294,-73.768378',
 '42.331666999999996,-89.15154799999998',
 '35.962317999999996,-79.97522099999998',
 '33.45,-112.07',
 '36.420403,-81.473438',
 '35.370461,-94.413065',
 '40.824806,-74.109314',
 '40.457812,-92.17129200000001',
 '40.288232,-74.617769',
 '42.522869,-75.57467700000002',
 '37.56,-77.45',
 '37.669746,-120.999103',
 '37.610182,-85.163472',
 '41.60185900000001,-72.65731600000002',
 '41.410849,-75.665902',
 '39.620378,-75.748778',
 '43.459514,-76.228818',
 '35.58754000000001,-88.859124',
 '36.154447999999995,-109.5526070000000

In [159]:
#Create new columns for the new data pulling from google api
no_missing_data['geocode_data'] = ''
no_missing_data['city'] = ''
no_missing_data['state'] = ''
no_missing_data['country'] = ''

#Create a function to call each url with the datas Lat Long
def reverse_geocode(latlng):
    #this going to store the url for each LatLng
    result = {}
    geocode_url = "https://maps.googleapis.com/maps/api/geocode/json?latlng="
    api_geo_details ="&key=" + gkey
    query_geo_url = geocode_url + latlng + api_geo_details
    geo_data_response = requests.get(query_geo_url)
    geo_data_json = geo_data_response.json()
    
    if len(geo_data_json['results']) > 0:
        result = geo_data_json['results'][0]
    return result

#use map to call the function for all column LatLng
no_missing_data['geocode_data'] = no_missing_data['LatLng'].map(reverse_geocode)
no_missing_data.head()

,Job Posting ID,Job Title,Category,Company Name,Location,Lat,Lng,LatLng,geocode_data,city,state,country
0,1312805093,Physician: Family Physician Job Opening in Ups...,Healthcare & Nursing Jobs,CompHealth,"Clayton, Jefferson County",44.239491,-76.085776,"44.239490999999994,-76.085776","{'address_components': [{'long_name': '512', '...",,,
1,1312805901,Physician: Urology Need in Western Ohio with L...,Healthcare & Nursing Jobs,CompHealth,"Ohio, US",40.305232,-82.824785,"40.305232000000004,-82.82478499999998","{'address_components': [{'long_name': '5629', ...",,,
2,1328241931,Physician: Internal Medicine Physician Job in ...,Healthcare & Nursing Jobs,CompHealth,"Cheshire, New Haven County",41.508367,-72.910620,"41.508367,-72.91062",{'address_components': [{'long_name': 'Cheshir...,,,
3,1334502463,Physician: A Locums Otolaryngologist Is Wanted...,Healthcare & Nursing Jobs,CompHealth,"Marion, Williamson County",37.730605,-88.933126,"37.730605,-88.933126","{'address_components': [{'long_name': '805', '...",,,
4,1312805719,"Physician: Hospitalist needed in Hartford, CT ...",Healthcare & Nursing Jobs,CompHealth,"Connecticut, US",41.601859,-72.657316,"41.60185900000001,-72.65731600000002","{'address_components': [{'long_name': '87', 's...",,,


In [160]:
#Created a function that searches for Country
def parse_country(geocode_data):
    if (not geocode_data is None) and ('address_components' in geocode_data):
        for component in geocode_data['address_components']:
            if 'country' in component['types']:
                return component['long_name']
    return None

#Created a function that searches for state
def parse_state(geocode_data):
    if (not geocode_data is None) and ('address_components' in geocode_data):
        for component in geocode_data['address_components']:
            if 'administrative_area_level_1' in component['types']:
                return component['long_name']
    return None

#Created a function that searches for city/town 
def parse_city(geocode_data):
    if (not geocode_data is None) and ('address_components' in geocode_data):
        for component in geocode_data['address_components']:
            if 'locality' in component['types']:
                return component['long_name']
            elif 'postal_town' in component['types']:
                return component['long_name']
            elif 'administrative_area_level_2' in component['types']:
                return component['long_name']
            elif 'administrative_area_level_1' in component['types']:
                return component['long_name']
    return None

#append these functions for city state and country using the maps function
no_missing_data['city'] = no_missing_data['geocode_data'].map(parse_city)
no_missing_data['state'] = no_missing_data['geocode_data'].map(parse_state)
no_missing_data['country'] = no_missing_data['geocode_data'].map(parse_country)
no_missing_data.head()

,Job Posting ID,Job Title,Category,Company Name,Location,Lat,Lng,LatLng,geocode_data,city,state,country
0,1312805093,Physician: Family Physician Job Opening in Ups...,Healthcare & Nursing Jobs,CompHealth,"Clayton, Jefferson County",44.239491,-76.085776,"44.239490999999994,-76.085776","{'address_components': [{'long_name': '512', '...",Clayton,New York,United States
1,1312805901,Physician: Urology Need in Western Ohio with L...,Healthcare & Nursing Jobs,CompHealth,"Ohio, US",40.305232,-82.824785,"40.305232000000004,-82.82478499999998","{'address_components': [{'long_name': '5629', ...",Sunbury,Ohio,United States
2,1328241931,Physician: Internal Medicine Physician Job in ...,Healthcare & Nursing Jobs,CompHealth,"Cheshire, New Haven County",41.508367,-72.910620,"41.508367,-72.91062",{'address_components': [{'long_name': 'Cheshir...,Cheshire,Connecticut,United States
3,1334502463,Physician: A Locums Otolaryngologist Is Wanted...,Healthcare & Nursing Jobs,CompHealth,"Marion, Williamson County",37.730605,-88.933126,"37.730605,-88.933126","{'address_components': [{'long_name': '805', '...",Marion,Illinois,United States
4,1312805719,"Physician: Hospitalist needed in Hartford, CT ...",Healthcare & Nursing Jobs,CompHealth,"Connecticut, US",41.601859,-72.657316,"41.60185900000001,-72.65731600000002","{'address_components': [{'long_name': '87', 's...",Cromwell,Connecticut,United States


In [161]:
#save raw data to dataRaw folder with no index
geo_raw_df = no_missing_data
geo_raw_df.to_csv(rawGeoData, index=False)
geo_raw_df.head()

,Job Posting ID,Job Title,Category,Company Name,Location,Lat,Lng,LatLng,geocode_data,city,state,country
0,1312805093,Physician: Family Physician Job Opening in Ups...,Healthcare & Nursing Jobs,CompHealth,"Clayton, Jefferson County",44.239491,-76.085776,"44.239490999999994,-76.085776","{'address_components': [{'long_name': '512', '...",Clayton,New York,United States
1,1312805901,Physician: Urology Need in Western Ohio with L...,Healthcare & Nursing Jobs,CompHealth,"Ohio, US",40.305232,-82.824785,"40.305232000000004,-82.82478499999998","{'address_components': [{'long_name': '5629', ...",Sunbury,Ohio,United States
2,1328241931,Physician: Internal Medicine Physician Job in ...,Healthcare & Nursing Jobs,CompHealth,"Cheshire, New Haven County",41.508367,-72.910620,"41.508367,-72.91062",{'address_components': [{'long_name': 'Cheshir...,Cheshire,Connecticut,United States
3,1334502463,Physician: A Locums Otolaryngologist Is Wanted...,Healthcare & Nursing Jobs,CompHealth,"Marion, Williamson County",37.730605,-88.933126,"37.730605,-88.933126","{'address_components': [{'long_name': '805', '...",Marion,Illinois,United States
4,1312805719,"Physician: Hospitalist needed in Hartford, CT ...",Healthcare & Nursing Jobs,CompHealth,"Connecticut, US",41.601859,-72.657316,"41.60185900000001,-72.65731600000002","{'address_components': [{'long_name': '87', 's...",Cromwell,Connecticut,United States
